# Elite Dangerous Tradings

Get the data from https://eddb.io/api

### Find the cheapest beer of the galaxy !!!

![](Elite Dangerous Tiles3.png)

![](Milky Way.png)

# 1. Load the Data

In [9]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

# get the application Id and print the tracking URL
applicationId = sc._jsc.sc().applicationId()
trackingUrl = 'http://lenovo02.localdomain:8088/proxy/'+applicationId

from IPython.core.display import display, HTML
display(HTML('<a href="'+trackingUrl+'">'+trackingUrl+'</a>'))

In [10]:
# Load stations
pathStations = "/user/mathias/stations.jsonl"
stations = sqlContext.read.json(pathStations)

# Load systems
pathSystems = "/user/mathias/systems_populated.jsonl"
systems = sqlContext.read.json(pathSystems)

# Load commodities
pathCommodities = "/user/mathias/commodities.json"
commodities = sqlContext.read.json(pathCommodities)
commodities = commodities.select(commodities.average_price, commodities.category.name, commodities.category_id, commodities.id, commodities.is_rare, commodities.name)

# Filter only Beer commodities
commoditiesBeers = commodities.filter((commodities.id == 8) | (commodities.id == 202))

# Load market prices
pathMarket = "/user/mathias/listings.csv"
market = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(pathMarket)

# Filter only Beer prices
beerMarket = market.filter((market.commodity_id == 8) | (market.commodity_id == 202))

In [11]:
# Display type of beers
table = ff.create_table(commoditiesBeers.limit(10).toPandas())
iplot(table, filename='commodities_table')

In [12]:
table = ff.create_table(beerMarket.limit(10).toPandas())
iplot(table, filename='market_table')

# 2. Cheapest beer of the Galaxy ?

In [18]:
# 1. filter only prices of stations with supplies
# 2. join market prices with commodities and stations
# 3. sort by buy price ascending
# 4. get only the top 100
topCheap = beerMarket\
    .filter(beerMarket.supply > 0)\
    .join(stations, beerMarket.station_id == stations.id)\
    .select(beerMarket.buy_price, stations.name)\
    .sort(beerMarket.buy_price.asc())\
    .limit(100)

# plot that
df1 = topCheap.toPandas()
trace1 = go.Scatter(
    x = df1['name'],
    y = df1['buy_price'],
    mode = 'lines',
    name = 'buy_price'
)

iplot([trace1], filename='beer-buy-price')

import os
with open(os.getcwd()+"/../dashboard/widgets/grid-2-1.html", "w") as text_file:
    text_file.write(plot([trace1], show_link=False, include_plotlyjs=False, output_type='div'))

![](DAG.png)

In [14]:
# same thing, but to find the best sell price
topSell = beerMarket\
    .filter(beerMarket.demand > 0)\
    .join(stations, beerMarket.station_id == stations.id)\
    .select(beerMarket.sell_price, stations.name)\
    .sort(beerMarket.sell_price.desc())\
    .limit(100)

# plot that
df2 = topSell.toPandas()
trace2 = go.Scatter(
    x = df2['name'],
    y = df2['sell_price'],
    mode = 'lines',
    name = 'sell_price'
)

iplot([trace2], filename='beer-sell-price')

import os
with open(os.getcwd()+"/../dashboard/widgets/grid-2-2.html", "w") as text_file:
    text_file.write(plot([trace2], show_link=False, include_plotlyjs=False, output_type='div'))

# 2. Objectif: Anaconda... comment farmer ?

![](Galnet-anaconda.jpg)

![](Anaconda-BluePrint.png)

Comment financer l'achat de ce vaiseau ??

On va "farmer" de la bière...

In [15]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType

# Bid
buy = beerMarket\
    .filter(beerMarket.supply > 0)\
    .repartition(5)\
    .alias("buy")

# Ask
sell = beerMarket\
    .filter(beerMarket.demand > 0)\
    .repartition(5)\
    .alias("sell")
    
computeProfit = udf(lambda supply, buy, demand, sell: min(demand, supply) * (sell - buy), IntegerType())
    
# compute profits between stations
profit = buy.join(sell)\
    .select(computeProfit(col("buy.supply"), col("buy.buy_price"), col("sell.demand"), col("sell.sell_price")).alias("profit"), 
            col("buy.station_id").alias("buy_station_id"), 
            col("sell.station_id").alias("sell_station_id"))\
    .filter(col("profit") > 0)
    
# save the result as files
#profit.write.mode("overwrite").parquet("profit.parquet1")

![](DAG2.png)

![](Parquet.png)

In [16]:
from pyspark.sql.functions import col, concat, concat_ws

# Get the stations
buy_stations = stations.alias("buy_station")
sell_stations = stations.alias("sell_station")

# Get top 10 profit between stations
profitParquet = sqlContext.read.parquet("profit.parquet")

df = profitParquet\
    .sort(profitParquet.profit.desc())\
    .limit(10)\
    .join(buy_stations, profitParquet.buy_station_id == col("buy_station.id"))\
    .join(sell_stations, profitParquet.sell_station_id == col("sell_station.id"))\
    .select(profitParquet.profit, concat_ws(" <-> ", col("buy_station.name"), col("sell_station.name")).alias("stations"))\
    .toPandas()

table = ff.create_table(df.sort_values(by=['profit'], ascending=[0]))
iplot(table, filename='profit_table')